In [92]:
import pandas as pd
import numpy as np

In [170]:
# reading main dataframe
df = pd.read_csv("../resources/TOPSIS_01.csv", sep=';', index_col=0, skiprows=3, header=None)
# resetting rows and column indexes
# df.reset_index(inplace=True, drop=True)
df.columns = range(df.shape[1])
df

,0,1,2,3,4,5,6,7
0,,,,,,,,
K-1,7,6,6,4,8,3,4,7
K-2,5,7,6,7,8,5,4,4
K-3,8,5,8,5,8,8,2,4
K-4,6,6,5,7,5,5,5,5
K-5,6,5,5,4,8,3,4,7
K-6,2,7,7,7,3,8,8,8
K-7,9,2,5,3,3,5,7,5
K-8,7,7,7,2,8,2,5,6
K-9,5,8,5,8,6,6,2,5


In [171]:
# reading weights as pandas Series
weights = pd.read_csv("../resources/TOPSIS_01.csv", sep=';', nrows=1, header=None, index_col=0, decimal=',').squeeze()
weights.reset_index(inplace=True, drop=True)
weights

0    0.10
1    0.10
2    0.10
3    0.10
4    0.24
5    0.03
6    0.12
7    0.21
Name: Waga (a), dtype: float64

In [172]:
# reading signs as pandas Series
signs = pd.read_csv("../resources/TOPSIS_01.csv", sep=';', skiprows=1, nrows=1, header=None, index_col=0, dtype='str').squeeze()
signs.reset_index(inplace=True, drop=True)
signs

0    +
1    +
2    +
3    -
4    +
5    +
6    +
7    +
Name: nan, dtype: object

In [173]:
# calculating squared root of sum of squares for each column
root_sum_squared = np.sqrt((df**2).sum(axis=0))
root_sum_squared

0    19.849433
1    19.026298
2    18.520259
3    17.492856
4    20.199010
5    16.431677
6    14.933185
7    17.916473
dtype: float64

In [174]:
# normalisation and multiplying each column by weight
weighted = (df / root_sum_squared) * weights
weighted

,0,1,2,3,4,5,6,7
0,,,,,,,,
K-1,0.035265,0.031535,0.032397,0.022866,0.095054,0.005477,0.032143,0.082047
K-2,0.025190,0.036791,0.032397,0.040016,0.095054,0.009129,0.032143,0.046884
K-3,0.040303,0.026279,0.043196,0.028583,0.095054,0.014606,0.016072,0.046884
K-4,0.030228,0.031535,0.026997,0.040016,0.059409,0.009129,0.040179,0.058605
K-5,0.030228,0.026279,0.026997,0.022866,0.095054,0.005477,0.032143,0.082047
K-6,0.010076,0.036791,0.037796,0.040016,0.035645,0.014606,0.064286,0.093768
K-7,0.045341,0.010512,0.026997,0.017150,0.035645,0.009129,0.056251,0.058605
K-8,0.035265,0.036791,0.037796,0.011433,0.095054,0.003651,0.040179,0.070326
K-9,0.025190,0.042047,0.026997,0.045733,0.071291,0.010954,0.016072,0.058605


In [175]:
# calculating A* (max if positive column else min) and A' (min if positive columne else max) for each column 
a_star = pd.Series([col.max() if signs[i]=='+' else col.min() for (i, col) in weighted.items()])
a_prime = pd.Series([col.min() if signs[i]=='+' else col.max() for (i, col) in weighted.items()])

In [176]:
# calculating S*
# subtracting A* from each column and squared
s_star_squared = (weighted - a_star) ** 2
# calculating sum for each row and square root
s_star = np.sqrt(s_star_squared.sum(axis=1))

In [177]:
# calculating S'
# subtracting A' from each column and squared
s_prime_squared = (weighted - a_prime) ** 2
# calculating sum for each row and square root
s_prime = np.sqrt(s_prime_squared.sum(axis=1))
s_prime

0
K-1     0.083002
K-2     0.070935
K-3     0.076385
K-4     0.048069
K-5     0.079509
K-6     0.076371
K-7     0.062921
K-8     0.087339
K-9     0.052923
K-10    0.027830
dtype: float64

In [188]:
# calculating CI
ci = s_prime/(s_star + s_prime)
ci = pd.DataFrame(ci)
ci.columns = ["CI"]

# creating candidate ranking - sorting descending by CI score
ranked = ci.sort_values(["CI"], ascending=False)
ranked

,CI
0,
K-8,0.699409
K-1,0.667252
K-5,0.633051
K-3,0.517039
K-2,0.510438
K-6,0.504045
K-7,0.445164
K-4,0.416461
K-9,0.406224


In [189]:
ranked.to_excel("../resources/ranking.xlsx")